## Using Tensorflow like Numpy

### Tensors and Operations

In [1]:
import tensorflow as tf

In [2]:
tf.constant([[1, 2, 3], [4, 5, 5]]) # matrix

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[1, 2, 3],
       [4, 5, 5]])>

In [3]:
tf.constant(40) # scalar

<tf.Tensor: shape=(), dtype=int32, numpy=40>

In [5]:
t = tf.constant([[1., 2., 3.], [4., 5., 6.]])
t.shape, t.dtype

(TensorShape([2, 3]), tf.float32)

In [6]:
t[:, 1:]

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[2., 3.],
       [5., 6.]], dtype=float32)>

In [7]:
t[..., 1, tf.newaxis]

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[2.],
       [5.]], dtype=float32)>

In [8]:
t + 10

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[11., 12., 13.],
       [14., 15., 16.]], dtype=float32)>

In [9]:
tf.square(t)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)>

In [10]:
t @ tf.transpose(t)

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[14., 32.],
       [32., 77.]], dtype=float32)>

#### Keras's Low Level API

In [11]:
K = tf.keras.backend
K.square(K.transpose(t)) + 10

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[11., 26.],
       [14., 35.],
       [19., 46.]], dtype=float32)>

### Tensors And NumPy

In [13]:
import numpy as np
a = np.array([2., 4., 5.])
tf.constant(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([2., 4., 5.])>

In [14]:
t.numpy()

array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)

In [15]:
tf.square(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 4., 16., 25.])>

In [16]:
np.square(t)

array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)

In [25]:
# Type Conversion

# tf.constant(2.) + tf.constant(40) # Gives Error
# tf.constant(2.) + tf.constant(40., dtype=tf.float64) # Also Give Error
tf.constant(2.0) + tf.cast(tf.constant(40.), dtype=tf.float32)

<tf.Tensor: shape=(), dtype=float32, numpy=42.0>

### Variables

In [26]:
# Tensors can't be changed so we use something called

v = tf.Variable([[1., 2., 3.], [4. ,5., 6.]])
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [27]:
# Modifying Tensors

v.assign(2 * v)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2.,  4.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [29]:
v.scatter_nd_update(indices=[[0, 0], [1, 2]], updates=[100., 200.])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[100.,   4.,   6.],
       [  8.,  10., 200.]], dtype=float32)>

### Other Data Structures
* Sparse tensors(tf.SparseTensor) -> efficiently represent tensors using mostly zeroes.
* Tensor Array(tf.TensorArray) -> lists of tensors. all have same shape and same type. Could Be Dynamic
* Ragged Tensor(tf.RaggedTensor) -> same as Tensor Array but Static
* String Tensor(tf.string) -> they represent byte strings. can be converted into Unicode
* Sets(tf.sets) -> can be represented by regular tensor or sparse tensor
* Queues (tf.queue)

## Customizing Models and Training Algorithms

### Custom Loss Functions